In [1]:
#Copyright April 1, 2018, Warren E. Agin
#Code released under the Creative Commons Attribution-NonCommercial-
#ShareAlike 4.0 International License. You may obtain a copy of the license at 
#https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode

In [1]:
import csv
import random
import os
import math
import numpy as np
import statistics as st

In [2]:
'''Script to generate training and test sets from Ch13TrimmedExtract.csv
Exports sets as comma delimited csv files, and an additional text file with information needed to regularize and normalize prediction data'''

#this section sets the initial variables to be used in the script

#set number of datapoints used for the training set and the test set
trainingSetSize=420000.00
testSetSize=30000.00

#columns used from the initial data set (first column is 0)
colsUsed =[4,6,19,20,23,31,37,49,50,53,54,59,60,69,71,84]   #last column of original set is 83 - 84 is for success

#initialize Readme file
readme=""

In [3]:
#this section opens the full data file, extracts the needed fields, identifies the size of the full data file,
#and extracts information needed to build the raw training and test sets

dataFile = open('CH13TrimmedExtract.csv', 'r+', newline='')
reader = csv.reader(dataFile, delimiter=',')

#determine number of datapoints in dataFile
dataSetSize = float(len(list(reader)))

readme = "Working with a data file of %s size. \r\n" % dataSetSize

In [4]:
#this section assigns lines to the raw training and test sets. Only data from the
#needed columns in the full data set is used.

#writes only the columns identified in the function
def write(writer, row, columns):
    revisedRow=[]
    for col in columns:
        revisedRow.append(row[col])
    writer.writerow(revisedRow)

successCount=0
failureCount=0
testCount=0

#open export files
trainingFile = open('rawTrainingFile.csv', 'w', newline='')
trainWriter = csv.writer(trainingFile)
testFile = open('rawTestFile.csv', 'w', newline='')
testWriter = csv.writer(testFile)
tempFile = open('tempFile.csv', 'w', newline='')
tempFile.close()
tempFile = open('tempFile.csv', 'r+', newline='')
tempWriter = csv.writer(tempFile)

#move rows to appropriate files
headerCount=0
dataFile.seek(0)
for row in reader:
    #header row added to top of training and test sets add success column at position
    if headerCount==0:
        row.append('SUCCESS')
        write(trainWriter, row, colsUsed)
        write(testWriter, row, colsUsed)
        headerCount += 1
        
    #for the remaining rows, calculate success as 1 or 0, append it
    #and write to test file, and then the training file on a 50/50 basis
    else:
        if row[69]=='13' and row[71] in ['A','B']:
            row.append(1)
        else:
            row.append(0)
        #fill out test set first
        if testCount<testSetSize:
            write(testWriter, row, colsUsed)
            testCount += 1

        #fill out training set with one half successes and one half failures
        elif successCount + failureCount< trainingSetSize:
            if successCount<(trainingSetSize/2) and row[84]==1:
                write(trainWriter, row, colsUsed)
                successCount += 1
            if failureCount<(trainingSetSize/2) and row[84]==0:
                write(trainWriter, row, colsUsed)
                failureCount += 1   
        #when training set is full end
        else:
            break
        
#log the results            

readme += "successCount: %s \r\n" % successCount
readme += "failureCount: %s \r\n" % failureCount
readme += "trainingSetSize: %s \r\n" % trainingSetSize
readme += "testCount: %s \r\n" % testCount
readme += "testSetSize: %s \r\n" % testSetSize    


In [5]:
#close the datafiles and delete the temp file
dataFile.close()
trainingFile.close()
testFile.close()
tempFile.close()
os.remove("tempFile.csv")

In [6]:
#this section loads the raw training file into a dictionary and extracts data regarding the training file needed to generate the features

#function to import the raw data files into a labeled dictionary
def importAsDict(filename):
    with open(filename, 'r+', newline='') as rawFile:
        rawReader = csv.reader(rawFile, delimiter=',')
        firstLine=0
        content=[]
        for row in rawReader:
            if firstLine==0:
                labels = row
                firstLine+=1
            else:
                content.append(row)
    content = np.ndarray.tolist(np.transpose(content))

    #create the dictionary - note the number of fields is hard-coded 
    rawDict = dict()
    for each in range(0,16):  #16 is the number of fields
        rawDict[labels[each]] = content[each]
    return(rawDict)

#import the training file into a labelled dictionary        
training = importAsDict('rawTrainingFile.csv')
    

In [7]:
#extract needed characteristics from training file

#compute medians for selected fields, excluding nulls, zeros and outliers where appropriate
#note that outliers are those values outside 0.13% of values (based on 3 standard deviation range)
def extractMedian(field, excludeZero, calcCutoff, cutoff): #excludeZero and calcCutoff are boolean
    temp = [x for x in training[field] if x!=""] #remove empty data from set
    temp = [float(x) for x in temp]
    if excludeZero:
        temp = [x for x in temp if x>0]
    if calcCutoff:
        temp.sort(reverse=True)
        cutoff=temp[round(float(len(temp))*.0013)]
    temp = [x for x in temp if x<float(cutoff)]
    median = st.median(temp)
    return(int(median), cutoff)

medianREALPROP, cutoffREALPROP = extractMedian('REALPROP',True,True,'0.0')
medianPERSPROP, cutoffPERSPROP = extractMedian('PERSPROP',False,True,'0.0')
medianUNSECPR, cutoffUNSECPR = extractMedian('UNSECPR',False,False,'336900.0') #336,900 was the unsecured debt limit for chapter 13 case in 2008
medianUNSECNPR, cutoffUNSECNPR = extractMedian('UNSECNPR',False,False,'336900.0')
medianAVGMNTHI, cutoffAVGMNTHI = extractMedian('AVGMNTHI',False,True,'0.0')
medianAVGMNTHE, cutoffAVGMNTHE = extractMedian('AVGMNTHE',False,True,'0.0')

readme += '\r\nThe median value and cutoff values for the following original data fields will be needed to calculate features for deployment.\r\n\r\n'
readme += 'median value for REALPROP = %s; cutoff = %s \r\n' % (medianREALPROP, cutoffREALPROP)
readme += 'median value for PERSPROP = %s; cutoff = %s \r\n' % (medianPERSPROP, cutoffPERSPROP)
readme += 'median value for UNSECPR = %s; cutoff = %s \r\n' % (medianUNSECPR, cutoffUNSECPR)
readme += 'median value for UNSECNPR = %s; cutoff = %s \r\n' % (medianUNSECNPR, cutoffUNSECNPR)
readme += 'median value for AVGMNTHI = %s; cutoff = %s \r\n' % (medianAVGMNTHI, cutoffAVGMNTHI)
readme += 'median value for AVGMNTHE = %s; cutoff = %s \r\n' % (medianAVGMNTHE, cutoffAVGMNTHE)

In [8]:
#cutoff size for districts - districts with a number of datapoints below the cutoff will not be used as features
#calculated using .95 confidence level, .05 confidence interval, .4 overall success rate, and trainingSetSize
districtCutoff = int(round(368.7936/(1+(.921984/(.0025*trainingSetSize)))))

#determine success ratio aggregated by district. Save in dictionary form for use in creating final training and test sets

#import file of district codes as ordered dictionary with codes as key and district names as content
with open('DistrictCodes.csv', 'r', newline='') as districtFile:
    reader = csv.DictReader(districtFile, delimiter=',')
    districtIndex = {}
    for row in reader:
        districtIndex=row

readme += '\r\nThe sample size cutoff for districts is %s and will be needed for deployment, in conjunction with the success rate for the total training set\r\n' % districtCutoff

In [10]:
#iterate over training data against districtIndex, count total of successes and failures in each district, calculate success ratio,
#and create new dictionary districtSuccess with district code as index and the success ratio for that district
#as the content.

#set default success rate and add to log
avgSuccessRatio = .40

readme += 'The success rate for the total training set is %s \r\n' % avgSuccessRatio

#build the districtSuccess dictionary. Where the number of data points is under the sample cutoff size,
#use the default success rate
undersamples=0
districtSuccess={}
for dist in districtIndex:
    successCount=0.0
    totalCount=0.0
    successRatio=0.0
    for item in range(0,int(trainingSetSize)):
        if str(dist)==str(training['DISTRICT'][item]):
            totalCount+=1
            if training['SUCCESS'][item]=="1":
                successCount+=1
    if successCount==0: successRatio=1.0
    if totalCount==0: totalCount=1.0
    if totalCount < districtCutoff:
        distSuccessRatio = avgSuccessRatio
        undersamples+=1
    else:
        distSuccessRatio=round(successCount/totalCount,4)
    districtSuccess[dist]=distSuccessRatio

#create csv file with district codes and district success rates for use in deploying model
with open('DistrictSuccessRates.csv', 'w', newline='') as f:  
    writer = csv.DictWriter(f, districtSuccess.keys(), delimiter=',')
    writer.writeheader()
    writer.writerow(districtSuccess)
    
readme += '\r\nA list of districts and their associated success ratios is saved in the file named DistrictSuccessRates.csv. %s districts had sample sizes under the cutoff size and were assigned the success rate for the total training set.\r\n' % undersamples

In [11]:
#This section transforms the raw training file into a the actual training file with the desired features
#note that the raw training file is already imported into training, and SUCCESS result added.

#function to assist in returning natural logs
def returnLog(x):
    return math.log(x if x>0 else 1)

def buildFeatures(rawFile, setSize):
    
    #initialize new feature lists 
    rawFile['DISTSUCCESS']=[]
    rawFile['FEEP']=[]
    rawFile['FEEI']=[]
    rawFile['FEEW']=[]
    rawFile['REALPROPNULL']=[]
    rawFile['REALPROPNONE']=[]
    rawFile['REALPROPVALUE']=[]
    rawFile['REALPROPVALUESQR']=[]
    rawFile['REALPROPVALUELOG']=[]
    rawFile['PERSPROPNULL']=[]
    rawFile['PERSPROPVALUE']=[]
    rawFile['PERSPROPVALUESQR']=[]
    rawFile['PERSPROPVALUELOG']=[]
    rawFile['UNSECNPRNULL']=[]
    rawFile['UNSECNPRVALUE']=[]
    rawFile['UNSECNPRVALUESQR']=[]
    rawFile['UNSECNPRVALUELOG']=[]
    rawFile['UNSECEXCESS']=[]
    rawFile['UNSECPRNULL']=[]
    rawFile['UNSECPRVALUE']=[]
    rawFile['UNSECPRVALUESQR']=[]
    rawFile['UNSECPRVALUELOG']=[]
    rawFile['AVGMNTHINULL']=[]
    rawFile['AVGMNTHIVALUE']=[]
    rawFile['AVGMNTHIVALUESQR']=[]
    rawFile['AVGMNTHIVALUELOG']=[]
    rawFile['IEINDEX']=[]
    rawFile['IEGAP']=[]
        
    for row in range(0,int(setSize)):
        
        #build DISTSUCCESS and delete DISTRICT
        if rawFile['DISTRICT'][row] in districtSuccess.keys():
            rawFile['DISTSUCCESS'].append(districtSuccess[rawFile['DISTRICT'][row]])
        else:
            rawFile['DISTSUCCESS'].append(avgSuccessRatio)
        
        #convert NTRDBT to integers
        if rawFile['NTRDBT'][row] == "b": rawFile['NTRDBT'][row]=int(1)
        else: rawFile['NTRDBT'][row]=int(0)
            
        #convert JOINT to integers
        if rawFile['JOINT'][row] == "y": rawFile['JOINT'][row]=int(1)
        else: rawFile['JOINT'][row]=int(0)
            
        #convert ORGD1FPRSE to integers
        if rawFile['ORGD1FPRSE'][row] == "y": rawFile['ORGD1FPRSE'][row]=int(1)
        else: rawFile['ORGD1FPRSE'][row]=int(0)  
            
        #convert PRFILE to integers
        if rawFile['PRFILE'][row] == "y": rawFile['PRFILE'][row]=int(1)
        else: rawFile['PRFILE'][row]=int(0)
            
        #build FEEP, FEEI, FEEW and delete ORGFEESTS
        if rawFile['ORGFEESTS'][row] == 'u': rawFile['FEEP'].append(int(1))
        else: rawFile['FEEP'].append(int(0))
        if rawFile['ORGFEESTS'][row] == 'i': rawFile['FEEI'].append(int(1))
        else: rawFile['FEEI'].append(int(0))     
        if rawFile['ORGFEESTS'][row] == 'w': rawFile['FEEW'].append(int(1))
        else: rawFile['FEEW'].append(int(0)) 
        
        #build REALPROPNULL, REALPROPNONE, REALPROPVALUE, REALPROPVALUESQR and REALPROPVALUELOG
        #and delete REALPROP
        if rawFile['REALPROP'][row] == "" or rawFile['REALPROP'][row]==None:
            rawFile['REALPROPNULL'].append(int(1))
            rawFile['REALPROPNONE'].append(int(0))
            rawFile['REALPROPVALUE'].append(medianREALPROP)
        elif rawFile['REALPROP'][row] == '0':
            rawFile['REALPROPNULL'].append(int(0))
            rawFile['REALPROPNONE'].append(int(1))
            rawFile['REALPROPVALUE'].append(0.0)
        elif float(rawFile['REALPROP'][row]) > cutoffREALPROP:
            rawFile['REALPROPNULL'].append(int(0))
            rawFile['REALPROPNONE'].append(int(0))
            rawFile['REALPROPVALUE'].append(medianREALPROP)
        else:
            rawFile['REALPROPNULL'].append(int(0))
            rawFile['REALPROPNONE'].append(int(0))
            rawFile['REALPROPVALUE'].append(float(rawFile['REALPROP'][row]))
        rawFile['REALPROPVALUESQR'].append(rawFile['REALPROPVALUE'][row]**2)
        rawFile['REALPROPVALUELOG'].append(round(returnLog(rawFile['REALPROPVALUE'][row]),4))
        
        #build PERSPROPNULL, PERSPROPVALUE, PERSPROPVALUESQR and PERSPROPVALUELOG
        #and delete PERSPROP
        
        if rawFile['PERSPROP'][row] == "" or rawFile['PERSPROP'][row]==None:
            rawFile['PERSPROPNULL'].append(int(1))
            rawFile['PERSPROPVALUE'].append(medianPERSPROP)
        elif float(rawFile['PERSPROP'][row]) > cutoffPERSPROP:
            rawFile['PERSPROPNULL'].append(int(0))
            rawFile['PERSPROPVALUE'].append(medianPERSPROP)
        else:
            rawFile['PERSPROPNULL'].append(int(0))
            rawFile['PERSPROPVALUE'].append(float(rawFile['PERSPROP'][row]))
        rawFile['PERSPROPVALUESQR'].append(rawFile['PERSPROPVALUE'][row]**2)
        rawFile['PERSPROPVALUELOG'].append(round(returnLog(rawFile['PERSPROPVALUE'][row]),4))
       
        #build UNSECNPRNULL, UNSECNPRVALUE, UNSECNPRVALUESQR and UNSECNPRVALUELOG
        #and delete UNSECNPR
        
        if rawFile['UNSECNPR'][row] == "" or rawFile['UNSECNPR'][row]==None:
            rawFile['UNSECNPRNULL'].append(int(1))
            rawFile['UNSECNPRVALUE'].append(medianUNSECNPR)
        elif float(rawFile['UNSECNPR'][row]) > float(cutoffUNSECNPR):
            rawFile['UNSECNPRNULL'].append(int(0))
            rawFile['UNSECNPRVALUE'].append(medianUNSECNPR)
        else:
            rawFile['UNSECNPRNULL'].append(int(0))
            rawFile['UNSECNPRVALUE'].append(float(rawFile['UNSECNPR'][row]))
        rawFile['UNSECNPRVALUESQR'].append(rawFile['UNSECNPRVALUE'][row]**2)
        rawFile['UNSECNPRVALUELOG'].append(round(returnLog(rawFile['UNSECNPRVALUE'][row]),4))       
         
        #build UNSECPRNULL, UNSECPRVALUE, UNSECPRVALUESQR and UNSECPRVALUELOG
        #and delete UNSECPR
        
        if rawFile['UNSECPR'][row] == "" or rawFile['UNSECPR'][row]==None:
            rawFile['UNSECPRNULL'].append(int(1))
            rawFile['UNSECPRVALUE'].append(medianUNSECPR)
        elif float(rawFile['UNSECPR'][row]) > float(cutoffUNSECPR):
            rawFile['UNSECPRNULL'].append(int(0))
            rawFile['UNSECPRVALUE'].append(medianUNSECPR)
        else:
            rawFile['UNSECPRNULL'].append(int(0))
            rawFile['UNSECPRVALUE'].append(float(rawFile['UNSECPR'][row]))
        rawFile['UNSECPRVALUESQR'].append(rawFile['UNSECPRVALUE'][row]**2)
        rawFile['UNSECPRVALUELOG'].append(round(returnLog(rawFile['UNSECPRVALUE'][row]),4))        
              
        #build UNSECEXCESS
        if rawFile['UNSECPR'][row] == "" or rawFile['UNSECPR'][row]==None:
            priority=0.0
        else: priority=float(rawFile['UNSECPR'][row])
        if rawFile['UNSECNPR'][row] == "" or rawFile['UNSECNPR'][row]==None:
            unsecured=0.0
        else: unsecured=float(rawFile['UNSECNPR'][row])
        if unsecured+priority > float(cutoffUNSECNPR):
            rawFile['UNSECEXCESS'].append(int(1))
        else: rawFile['UNSECEXCESS'].append(int(0))
        
        #build AVGMNTHINULL, AVGMNTHIVALUE, AVGMNTHIVALUESQR and AVGMNTHIVALUELOG
        #and delete AVGMNTHI
        
        if rawFile['AVGMNTHI'][row] == "" or rawFile['AVGMNTHI'][row]==None:
            rawFile['AVGMNTHINULL'].append(int(1))
            rawFile['AVGMNTHIVALUE'].append(medianAVGMNTHI)
        elif float(rawFile['AVGMNTHI'][row]) > float(cutoffAVGMNTHI):
            rawFile['AVGMNTHINULL'].append(int(0))
            rawFile['AVGMNTHIVALUE'].append(medianAVGMNTHI)
        else:
            rawFile['AVGMNTHINULL'].append(int(0))
            rawFile['AVGMNTHIVALUE'].append(float(rawFile['AVGMNTHI'][row]))
        rawFile['AVGMNTHIVALUESQR'].append(rawFile['AVGMNTHIVALUE'][row]**2)
        rawFile['AVGMNTHIVALUELOG'].append(round(returnLog(rawFile['AVGMNTHIVALUE'][row]),4))  
        
        #build IENDEX - uses AVGMNTHIVALUE from above, and expense
        if rawFile['AVGMNTHE'][row] == "" or rawFile['AVGMNTHE'][row]==None:
            expense =float(medianAVGMNTHI)
        elif float(rawFile['AVGMNTHE'][row]) > float(cutoffAVGMNTHE):
            expense =float(medianAVGMNTHI)
        else: expense = float(rawFile['AVGMNTHE'][row])    
        rawFile['IEINDEX'].append(round(float(max(.1,float(rawFile['AVGMNTHIVALUE'][row]))/max(.1,expense)),4))
        
        #build IEGAP - uses AVGMNTHIVALUE from above, and expense
        rawFile['IEGAP'].append(float(rawFile['AVGMNTHIVALUE'][row])-(expense))
    
    #delete depreciated fields
    for each in ['CASEKEY','CLCHPT','D1FDSP','DISTRICT','ORGFEESTS','REALPROP','PERSPROP','UNSECNPR','UNSECPR','AVGMNTHI','AVGMNTHE']: 
        if each in rawFile:
            del rawFile[each]

    return(rawFile)

#run buildFeatures to transform the training file
training = buildFeatures(training, trainingSetSize)

In [ ]:
#write the column features into a csv file for use by the learner and in deployment, along with a csv with the names of the column features
with open('featureNames.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, training.keys(), delimiter=',')
    writer.writeheader()


def writeFeatures(file, filename, setSize):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for row in range(0,int(setSize)):
            set=[]
            for key in file.keys():
                set.append(str(file[key][row]))
            writer.writerow(set)
    return()

writeFeatures(training, 'trainingFile.csv', trainingSetSize)

In [13]:
#repeat the steps using the prepared functions to import the raw test data, convert it to include the required features,
#and write the final test file for use by the learner

test = importAsDict('rawTestFile.csv')
#test = addSuccess(test, testSetSize)
test = buildFeatures(test, testSetSize)
writeFeatures(test, 'testFile.csv', testSetSize)

readme += '\r\nThe training data file is named trainingFile.csv, the test data file is named testData.csv, and an ordered list of feature names is in the file named featureNames.csv\r\n'

In [14]:
#delete the raw files

os.remove('rawTrainingFile.csv')
os.remove('rawTestFile.csv')

In [15]:
#write the readme file to LEARNER_README.txt

with open('LEARNER_README.txt', 'w', newline='') as f:
    writer = csv.writer(f)
    readme=[readme]
    writer.writerow(readme)